<a href="https://colab.research.google.com/github/dhnanjay/SEC/blob/main/step2_prep_data_finetuningipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 0. Setup

In [ ]:
!pip install huggingface_hub datasets --quiet
https://twitter.com/virattt/status/1793040762025791870

In [ ]:
!huggingface-cli login

### 1. Load dataset

In [ ]:
from datasets import load_dataset

finetuning_dataset = load_dataset("virattt/llama-3-8b-financialQA", split="train")

In [ ]:
finetuning_dataset

Dataset({
    features: ['question', 'answer', 'context'],
    num_rows: 100
})

### 2. Tokenize the dataset

In [ ]:
from transformers import AutoTokenizer

model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
def tokenize_fn(examples):
    # Create the prompt text
    text = f"Question: {examples['question'][0]} Answer: {examples['answer'][0]}"

    # Add pad token
    tokenizer.pad_token = tokenizer.eos_token

    # Tokenize the text
    return tokenizer(
        text,
        return_tensors="np",
        truncation=True,
        max_length=4096,
    )

In [ ]:
tokenized_dataset = finetuning_dataset.map(
    tokenize_fn,
    batched=True,
    batch_size=1,
    drop_last_batch=True
)

print(tokenized_dataset)

In [ ]:
tokenized_dataset = tokenized_dataset.add_column("labels", tokenized_dataset["input_ids"])

### 3. Create train/test splits

In [ ]:
split_dataset = tokenized_dataset.train_test_split(test_size=0.1, shuffle=True, seed=42)

In [ ]:
print(split_dataset)

DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'context', 'input_ids', 'attention_mask'],
        num_rows: 90
    })
    test: Dataset({
        features: ['question', 'answer', 'context', 'input_ids', 'attention_mask'],
        num_rows: 10
    })
})
